# ENGIE Wind Power Prediction  
## Notebook 01 — Load, Merge and Data Quality Checks

Goal of this notebook:
- Load ENGIE sensor data (X) and target data (Y)
- Merge them using ID
- Perform sanity and quality checks before modeling
- Save a clean dataset for the next notebooks

### 1️ Environment Setup

We define the paths to the raw datasets stored in the `data/` folder.


In [1]:
import os
import pandas as pd
import numpy as np

DATA_DIR = "../data"  # because notebook is inside /notebooks
X_PATH = os.path.join(DATA_DIR, "engie_X.csv")
Y_PATH = os.path.join(DATA_DIR, "engie_Y.csv")

print("X exists:", os.path.exists(X_PATH), "|", X_PATH)
print("Y exists:", os.path.exists(Y_PATH), "|", Y_PATH)


X exists: True | ../data\engie_X.csv
Y exists: True | ../data\engie_Y.csv


### 2️ Load Target Data (Y)

The file `engie_Y.csv` contains the target variable:

- `ID`: unique identifier of each observation  
- `TARGET`: active power produced by a wind turbine at a given timestamp


In [2]:
Y = pd.read_csv(Y_PATH, sep=";")
print("Y shape:", Y.shape)
display(Y.head())

assert set(Y.columns) == {"ID", "TARGET"}
assert Y["ID"].is_unique
assert Y["TARGET"].isna().mean() < 0.001

print("✅ Y loaded correctly")


Y shape: (617386, 2)


,ID,TARGET
0,1,-0.703
1,2,-0.747
2,3,-0.791
3,4,-0.736
4,5,-1.055


✅ Y loaded correctly


### 3️ Load Sensor Features (X)

The file `engie_X.csv` contains the explanatory variables (sensor measurements) recorded every 10 minutes for each wind turbine.

Key columns include:

- `ID`: unique identifier linking to the target file  
- `MAC_CODE`: wind turbine identifier (WT1, WT2, WT3, WT4)  
- `Date_time`: timestamp index  
- Multiple sensor features (temperatures, rotational speeds, electrical variables, orientations, etc.)


In [3]:
X = pd.read_csv(X_PATH, sep=";")
print("X shape:", X.shape)
display(X.head())

required = {"ID", "MAC_CODE", "Date_time"}
assert required.issubset(set(X.columns))
assert X["ID"].is_unique
assert X.shape[0] > 10000

print("MAC_CODE unique:", sorted(X["MAC_CODE"].unique()))
print("✅ X loaded correctly")


X shape: (617386, 78)


,ID,MAC_CODE,Date_time,Pitch_angle,Pitch_angle_min,Pitch_angle_max,Pitch_angle_std,Hub_temperature,Hub_temperature_min,Hub_temperature_max,...,Rotor_speed,Rotor_speed_min,Rotor_speed_max,Rotor_speed_std,Rotor_bearing_temperature,Rotor_bearing_temperature_min,Rotor_bearing_temperature_max,Rotor_bearing_temperature_std,Absolute_wind_direction_c,Nacelle_angle_c
0,1,WT3,1.0,92.470001,92.470001,92.470001,0.0,7.00,7.0,7.0,...,0.0,0.0,0.0,0.0,2.4,2.4,2.4,0.0,294.19000,294.23999
1,2,WT3,2.0,92.470001,92.470001,92.470001,0.0,7.00,7.0,7.0,...,0.0,0.0,0.0,0.0,2.4,2.4,2.4,0.0,297.82999,294.23999
2,3,WT3,3.0,92.470001,92.470001,92.470001,0.0,7.00,7.0,7.0,...,0.0,0.0,0.0,0.0,2.4,2.4,2.4,0.0,322.20999,294.23999
3,4,WT3,4.0,92.470001,92.470001,92.470001,0.0,6.97,6.7,7.0,...,0.0,0.0,0.0,0.0,2.4,2.4,2.4,0.0,318.69000,294.23999
4,5,WT3,5.0,92.470001,92.470001,92.470001,0.0,6.93,6.0,7.0,...,0.0,0.0,0.0,0.0,2.4,2.4,2.5,0.0,314.89001,294.23999


MAC_CODE unique: ['WT1', 'WT2', 'WT3', 'WT4']
✅ X loaded correctly


### 4️ Merge X and Y

We merge the sensor data and the target using the unique identifier `ID`.


In [4]:
df = X.merge(Y, on="ID", how="inner")
print("Merged df shape:", df.shape)
display(df.head())

coverage = df.shape[0] / Y.shape[0]
print(f"Coverage vs Y: {coverage:.2%}")
assert coverage > 0.98

print("✅ Merge successful")


Merged df shape: (617386, 79)


,ID,MAC_CODE,Date_time,Pitch_angle,Pitch_angle_min,Pitch_angle_max,Pitch_angle_std,Hub_temperature,Hub_temperature_min,Hub_temperature_max,...,Rotor_speed_min,Rotor_speed_max,Rotor_speed_std,Rotor_bearing_temperature,Rotor_bearing_temperature_min,Rotor_bearing_temperature_max,Rotor_bearing_temperature_std,Absolute_wind_direction_c,Nacelle_angle_c,TARGET
0,1,WT3,1.0,92.470001,92.470001,92.470001,0.0,7.00,7.0,7.0,...,0.0,0.0,0.0,2.4,2.4,2.4,0.0,294.19000,294.23999,-0.703
1,2,WT3,2.0,92.470001,92.470001,92.470001,0.0,7.00,7.0,7.0,...,0.0,0.0,0.0,2.4,2.4,2.4,0.0,297.82999,294.23999,-0.747
2,3,WT3,3.0,92.470001,92.470001,92.470001,0.0,7.00,7.0,7.0,...,0.0,0.0,0.0,2.4,2.4,2.4,0.0,322.20999,294.23999,-0.791
3,4,WT3,4.0,92.470001,92.470001,92.470001,0.0,6.97,6.7,7.0,...,0.0,0.0,0.0,2.4,2.4,2.4,0.0,318.69000,294.23999,-0.736
4,5,WT3,5.0,92.470001,92.470001,92.470001,0.0,6.93,6.0,7.0,...,0.0,0.0,0.0,2.4,2.4,2.5,0.0,314.89001,294.23999,-1.055


Coverage vs Y: 100.00%
✅ Merge successful


In [5]:
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
display(missing_pct.head(20))


Grid_voltage                        16.411451
Grid_voltage_min                    16.411451
Grid_voltage_max                    16.411451
Grid_voltage_std                    16.411451
Gearbox_inlet_temperature            1.306152
Generator_converter_speed            1.306152
Gearbox_inlet_temperature_max        1.306152
Gearbox_inlet_temperature_min        1.306152
Generator_converter_speed_std        1.306152
Generator_converter_speed_max        1.306152
Generator_converter_speed_min        1.306152
Gearbox_inlet_temperature_std        1.306152
Nacelle_angle_c                      0.011662
Absolute_wind_direction_c            0.011662
Rotor_speed_min                      0.000000
Absolute_wind_direction              0.000000
Gearbox_oil_sump_temperature_min     0.000000
Gearbox_oil_sump_temperature_max     0.000000
Gearbox_oil_sump_temperature_std     0.000000
Nacelle_angle                        0.000000
dtype: float64

### 5 Turbine Distribution & Time Coverage

We verify:
- Number of records per turbine
- Time range of the dataset
- Basic statistics of the target variable


In [6]:
print("Rows per turbine:")
display(df["MAC_CODE"].value_counts())

print("Date_time range:", df["Date_time"].min(), "→", df["Date_time"].max())

df["TARGET"].describe()


Rows per turbine:


MAC_CODE
WT2    154791
WT1    154707
WT3    154253
WT4    153635
Name: count, dtype: int64

Date_time range: 1.0 → 157680.0


count    617386.000000
mean        372.752158
std         468.001341
min         -19.479999
25%          18.624000
50%         193.985989
75%         540.684000
max        2256.057110
Name: TARGET, dtype: float64

In [7]:
# Temporal Consistency Check
df_sorted = df.sort_values(["MAC_CODE", "Date_time"])
diffs = df_sorted.groupby("MAC_CODE")["Date_time"].diff().dropna()

print("Date_time diff min/max:", diffs.min(), diffs.max())
display(diffs.value_counts().head(10))


Date_time diff min/max: 0.0 991.0


Date_time
1.0      617031
2.0         125
0.0          72
7.0          19
3.0          11
8.0           6
11.0          5
4.0           5
433.0         4
50.0          4
Name: count, dtype: int64

### 6 Save Clean Dataset

We export the merged and validated dataset for use in modeling notebooks.


In [8]:
df.to_parquet(os.path.join(DATA_DIR, "engie_full.parquet"), index=False)
print("✅ engie_full.parquet saved")


✅ engie_full.parquet saved
